## Setup

In [1]:
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf

import tensorflow_datasets as tfds
tfds.disable_progress_bar()

from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

import matplotlib.pyplot as plt
import numpy as np

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPUs will likely run quickly with dtype policy mixed_float16 as they all have compute capability of at least 7.0


In [2]:
try: 
    tpu = None
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver() 
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError: 
    strategy = tf.distribute.MirroredStrategy() 

print("Number of accelerators: ", strategy.num_replicas_in_sync)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Number of accelerators:  4


## Define hyperparameters

In [3]:
TARGET_DIM = (224, 224)

AUTO = tf.data.AUTOTUNE
BATCH_SIZE = 128 * strategy.num_replicas_in_sync
EPOCHS = 10

## Prepare data loaders

In [4]:
train_ds, validation_ds, test_ds = tfds.load(
    "cats_vs_dogs",
    # Reserve 10% for validation and 10% for test
    split=["train[:90%]", "train[90%:95%]", "train[95%:]"],
    as_supervised=True,  
)

In [5]:
def preprocess_dataset(image, label):
    image = tf.image.resize(image, (TARGET_DIM[0], TARGET_DIM[1]))
    label = tf.one_hot(label, depth=2)
    return (image, label)

In [6]:
train_ds = (
    train_ds
    .shuffle(BATCH_SIZE * 100)
    .map(preprocess_dataset, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)
validation_ds = (
    validation_ds
    .map(preprocess_dataset, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)
test_ds = (
    test_ds
    .map(preprocess_dataset, num_parallel_calls=AUTO)
    .batch(BATCH_SIZE)
    .prefetch(AUTO)
)

## Build, train, and evaluate model

In [7]:
def get_model():
    backbone = tf.keras.applications.DenseNet121(weights=None, include_top=True, classes=2)
    backbone.trainable = True
    
    inputs = layers.Input((TARGET_DIM[0], TARGET_DIM[1], 3))
    x = layers.experimental.preprocessing.Rescaling(scale=1./255)(inputs)
    outputs = backbone(x)
    outputs = layers.Activation("linear", dtype="float32")(outputs)

    return tf.keras.Model(inputs, outputs)

In [8]:
# We will use this initial random model to train both the models
# for reproducibility
initial_model = get_model()
initial_model.save("initial_model")

INFO:tensorflow:Assets written to: initial_model/assets


INFO:tensorflow:Assets written to: initial_model/assets


In [10]:
with strategy.scope():
    model = tf.keras.models.load_model("initial_model")
    model.compile(loss=keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
                optimizer="sgd",
                metrics=["accuracy"])
model.fit(train_ds,
          validation_data=validation_ds,
          epochs=EPOCHS)

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).


...

Epoch 1/10
INFO:tensorflow:batch_all_reduce: 364 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 364 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 364 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 364 all-reduces with algorithm = nccl, num_packs = 1


41/41 [==============================] - 207s 730ms/step - loss: 0.6687 - accuracy: 0.5978 - val_loss: 0.6931 - val_accuracy: 0.5047
Epoch 2/10
41/41 [==============================] - 20s 388ms/step - loss: 0.6293 - accuracy: 0.6613 - val_loss: 0.6932 - val_accuracy: 0.5047
Epoch 3/10
41/41 [==============================] - 19s 381ms/step - loss: 0.6194 - accuracy: 0.6739 - val_loss: 0.7006 - val_accuracy: 0.4953
Epoch 4/10
41/41 [==============================] - 19s 380ms/step - loss: 0.6082 - accuracy: 0.6937 - val_loss: 0.7164 - val_accuracy: 0.4953
Epoch 5/10
41/41 [==============================] - 19s 381ms/step - loss: 0.5956 - accuracy: 0.7067 - val_loss: 0.7360 - val_accuracy: 0.4953
Epoch 6/10
41/41 [==============================] - 19s 384ms/step - loss: 0.5941 - accuracy: 0.7115 - val_loss: 0.7672 - val_accuracy: 0.4953
Epoch 7/10
41/41 [==============================] - 19s 382ms/step - loss: 0.5813 - accuracy: 0.7242 - val_loss: 0.9797 - val_accuracy: 0.4953
Epoch 8/1

In [11]:
with strategy.scope():
    _, test_acc = model.evaluate(test_ds)
print("Test accuracy: {:.2f}%".format(test_acc * 100))

model.save("standard_densenet_model")

3/3 [==============================] - 1s 127ms/step - loss: 1.1822 - accuracy: 0.5030
Test accuracy: 50.30%
INFO:tensorflow:Assets written to: standard_densenet_model/assets


INFO:tensorflow:Assets written to: standard_densenet_model/assets
